# Python + Redis - Jak to działa?

Pierwszym krokiem wymaganym do połączenia Python'a z Redisem jest zainstalowanie biblioteki `redis-py`:

W terminalu wywołaj polecenie:
```
pip install redis

```

---
## Importowanie i łączenie się z Redisem:

In [2]:
import redis

r = redis.Redis(host='localhost', port=6379, db=0)

"""
Typowym portem Redis przy instalacji lokalnej jest 6379, a domyślną bazą danych jest 0.
"""

print(r.ping())  # Sprawdzenie połączenia z Redisem

True


---
## Podstawowe operacje:

In [5]:
import redis as r

r = r.Redis(host='localhost', port = 6379, db =0)
#-------------------------------------------------------
def oddzielenie():
    print("-"  * 50)
#-------------------------------------------------------

#Ustawianie wartości klucz - wartość:
r.set('samochod_1', 'BMW')
r.set('samochod_2', 'Audi')
r.set('samochod_3', 'Mercedes')

#-------------------------------------------------------

#Odczytywanie wartości klucz - wartość:
samochod = r.get('samochod_1')
#-------------------------------------------------------

"""
Uwaga!
Redis zwraca wartośći w formacie binarnym!
Przy wyświetlaniu, korzystaniu z danych musimy je zdekodować./
"""

print(f"Samochód: {samochod.decode()}") #Odczyt z dekodowaniem
oddzielenie()
#-------------------------------------------------------

#Usuwanie wartości klucz - wartość:
r.delete('samochod_1')
#-------------------------------------------------------

#Sprawdzanie czy klucz istnieje:
print("Istnieje!" if r.exists('samochod_1') else "Nie istnieje!")
print("Istnieje!" if r.exists('samochod_2') else "Nie istnieje!")
oddzielenie()
#-------------------------------------------------------

#Spradzanie typu klucza:
print(r.type('samochod_2')) #Odczyt typu klucza
oddzielenie()
#-------------------------------------------------------

#Losowanie klucza (z całej bazy danych):
print(f"Wylosowano: {r.randomkey()}")
oddzielenie()
#-------------------------------------------------------

#Pobieranie zserializowanych danych:
print(r.dump('samochod_2')) #Odczyt zserializowanych danych
oddzielenie()
#-------------------------------------------------------

#Pobieranie kluczy według wzorca:
print(r.keys('samochod_*')) #Odczyt kluczy według wzorca
oddzielenie()

Samochód: BMW
--------------------------------------------------
Nie istnieje!
Istnieje!
--------------------------------------------------
b'string'
--------------------------------------------------
Wylosowano: b'pracownik_6'
--------------------------------------------------
b'\x00\x04Audi\n\x001{\xbf\x1d\xaa\xc9\xaa*'
--------------------------------------------------
[b'samochod_2', b'samochod_3']
--------------------------------------------------


---
## Zarządzanie czasem życia kluczy:

In [7]:
import redis as r

r = r.Redis('localhost', port = 6379, db =0)

#-------------------------------------------------------
def oddzielenie():
    print("-"  * 50)
#-------------------------------------------------------
r.set('odjazd_1', 'Warszawa')


#Ustawianie czasu życia dla istniejącego klucza (w seknduach):
r.expire('odjazd_1', 600) #Ustawienie czasu życia klucza (odjazd za 10 minut)
#-------------------------------------------------------

#Pobieranie czasu życia klucza:
print(f"Odjazd za: {r.ttl('odjazd_1')} sekund")
print(f"Odjazd za: {r.pttl('odjazd_1')} milisekund")

print(f"Odjazd za: {r.ttl('odjazd_1') / 60} minut")
oddzielenie()
#-------------------------------------------------------

#Usuwanie czasu życia klucza:
r.persist('odjazd_1') #Usunięcie czasu życia klucza
#-------------------------------------------------------

Odjazd za: 600 sekund
Odjazd za: 599996 milisekund
Odjazd za: 10.0 minut
--------------------------------------------------


True

---
## Operacje na ciągach znaków:

In [13]:
"""
Uwaga!
Redis nie obsługuje typów danych takich jak int, float, bool
Podstawowym typem danych jest string.
Wszystkie inne typy danych są konwertowane na string.

Jako, że jest to podstawowy typ danych, niektóre z poleceń mogą się powtarzać.
"""

import redis as r

r = r.Redis('localhost', port = 6379, db =0)

#-------------------------------------------------------
def oddzielenie():
    print("-"  * 50)

#-------------------------------------------------------
wiersz1  = "Uciekałem przez sen w nocy,\n Mając skrzydła ku pomocy,\n Lecz mię miłość poimała,\n Choć na nogach ołów miała.\n Hanno, co to znamionuje?\n"
wiersz2 = "Podobno mi praktykuje,\n Że ja, będąc uwikłany\n Tymi i owymi pany,\n Wszytkich inszych łatwie zbędę -\n Tobie służyć wiecznie będę."
osoba = "Jan Kochanowski"

r.set("autor1", osoba)
r.set("wiersz1", wiersz1)

#-------------------------------------------------------
#Pobieranie fragmentu tekstu:
print(f"Fragment tekstu: {r.getrange('wiersz1', 0, 20).decode()}")
oddzielenie()

#-------------------------------------------------------
#Pobieranie długości tekstu:
print(f"Długość tekstu: {r.strlen('wiersz1')}")
oddzielenie()

#-------------------------------------------------------
#Łączenie tekstu:
r.append('wiersz1', wiersz2) #Łączenie tekstu
print(f"Wiersz: \n{r.get('wiersz1').decode()}")
oddzielenie()

#-------------------------------------------------------
#Zamiana fragmentu tekstu:
r.setrange('autor1', 0, 'Jan Brzechwa   ')

print(f"Autor: {r.get('autor1').decode()}")
oddzielenie()

Fragment tekstu: Uciekałem przez sen 
--------------------------------------------------
Długość tekstu: 147
--------------------------------------------------
Wiersz: 
Uciekałem przez sen w nocy,
 Mając skrzydła ku pomocy,
 Lecz mię miłość poimała,
 Choć na nogach ołów miała.
 Hanno, co to znamionuje?
Podobno mi praktykuje,
 Że ja, będąc uwikłany
 Tymi i owymi pany,
 Wszytkich inszych łatwie zbędę -
 Tobie służyć wiecznie będę.
--------------------------------------------------
Autor: Jan Brzechwa   
--------------------------------------------------


---
## Polecenia działające na liczbach:

In [19]:
import redis as r
r = r.Redis('localhost', port=6379, db=0)

#-------------------------------------------------------
def oddzielenie():
    print("-" * 50)

#-------------------------------------------------------
r.mset({'a': 7, 'b': 2.5, 'c': 2050, 'd': 21320})

#-------------------------------------------------------
# Inkrementacja wartości:
print(f"Przed inkrementacją: {r.get('a').decode()}")
r.incr('a')
r.incr('a')
print(f"Po inkrementacji: {r.get('a').decode()}")
oddzielenie()

#-------------------------------------------------------
# Dekrementacja wartości:
print(f"Przed dekrementacją: {r.get('d').decode()}")
for _ in range(8):
    r.decr('d')  #
print(f"Po dekrementacji: {r.get('d').decode()}")
oddzielenie()

#-------------------------------------------------------
# Zwiększanie wartości o zadany krok:
print(f"Przed zwiększeniem: {r.get('b').decode()}")
r.incrbyfloat('b', 2.5)
print(f"Po zwiększeniu: {r.get('b').decode()}")
oddzielenie()

#-------------------------------------------------------
# Zmniejszanie wartości o zadany krok:
print(f"Przed zmniejszeniem: {r.get('c').decode()}")
r.decrby('c', 25)
print(f"Po zmniejszeniu: {r.get('c').decode()}")
oddzielenie()

#-------------------------------------------------------
print(f"Dzień: {r.get('a').decode()}")
print(f"Miesiąc: {r.get('b').decode()}")
print(f"Rok: {r.get('c').decode()}")
print(f"Indeks: {r.get('d').decode()}")


Przed inkrementacją: 7
Po inkrementacji: 9
--------------------------------------------------
Przed dekrementacją: 21320
Po dekrementacji: 21312
--------------------------------------------------
Przed zwiększeniem: 2.5
Po zwiększeniu: 5
--------------------------------------------------
Przed zmniejszeniem: 2050
Po zmniejszeniu: 2025
--------------------------------------------------
Dzień: 9
Miesiąc: 5
Rok: 2025
Indeks: 21312


---
## Działania na listach:

In [25]:
import redis as r

r = r.Redis('localhost', port=6379, db=0)

#-------------------------------------------------------

def oddzielenie():
    print("-" * 50)

#-------------------------------------------------------
#Usuwanie listy jeżeli ta istnieje:
r.delete('owoce')

#-------------------------------------------------------
#Tworzenie listy:
r.rpush("owoce", "Jabłko", 'Banan', 'Gruszka', 'Ananas')

#-------------------------------------------------------
#Pobieranie całej listy:
owoce = r.lrange('owoce', 0, -1)

for owoc in owoce:
    print(f"-> {owoc.decode()}")
oddzielenie()

#-------------------------------------------------------
# Usuwanie i pobieranie pierwszego elementu z listy (lewa strona)
pierwszy = r.lpop('owoce')
print(f"Usunięto pierwszy owoc: {pierwszy.decode()}")
oddzielenie()

#-------------------------------------------------------
# Usuwanie i pobieranie ostatniego elementu z listy (prawa strona)
ostatni = r.rpop('owoce')
print(f"Usunięto ostatni owoc: {ostatni.decode()}")
oddzielenie()

#-------------------------------------------------------
# Wyświetlenie aktualnej zawartości listy po usunięciach
print("Aktualna lista owoców po usunięciach:")
owoce = r.lrange('owoce', 0, -1)
for owoc in owoce:
    print(f"-> {owoc.decode()}")
oddzielenie()

#-------------------------------------------------------
# Sprawdzenie długości listy
dlugosc = r.llen('owoce')
print(f"Liczba owoców na liście: {dlugosc}")
oddzielenie()

#-------------------------------------------------------
# Zamiana elementu na konkretnym indeksie
r.lset('owoce', 0, 'Mango')
print("Po zamianie pierwszego elementu na 'Mango':")
owoce = r.lrange('owoce', 0, -1)
for owoc in owoce:
    print(f"-> {owoc.decode()}")
oddzielenie()

#-------------------------------------------------------
# Usuwanie wszystkich wystąpień 'Banan' z listy
r.lrem('owoce', 0, 'Banan')
print("Po usunięciu wszystkich 'Bananów':")
owoce = r.lrange('owoce', 0, -1)
for owoc in owoce:
    print(f"-> {owoc.decode()}")
oddzielenie()


-> Jabłko
-> Banan
-> Gruszka
-> Ananas
--------------------------------------------------
Usunięto pierwszy owoc: Jabłko
--------------------------------------------------
Usunięto ostatni owoc: Ananas
--------------------------------------------------
Aktualna lista owoców po usunięciach:
-> Banan
-> Gruszka
--------------------------------------------------
Liczba owoców na liście: 2
--------------------------------------------------
Po zamianie pierwszego elementu na 'Mango':
-> Mango
-> Gruszka
--------------------------------------------------
Po usunięciu wszystkich 'Bananów':
-> Mango
-> Gruszka
--------------------------------------------------


---
## Działania na Hash'ach:

In [40]:
import redis as r

r = r.Redis('localhost', port=6379, db=0)

#-------------------------------------------------------
def oddzielenie():
    print("-" * 50)

#-------------------------------------------------------
#Ustawianie pół w hash:
r.hset('pc_1', mapping={
    'marka' : 'Lenovo',
    'model' : 'TinkPad',
    'rok' : 2024,
})

#-------------------------------------------------------
#Pobieranie wszysytkich pól i wartości:
dane = r.hgetall('pc_1')
print("Dane PC1: ")
for k, v in dane.items():
    print(f"{k.decode()} : {v.decode()}")
oddzielenie()

#-------------------------------------------------------
#Pobieranie konkretnego pola:
marka = r.hget('pc_1', 'marka')
print(f"Marka PC1: {marka.decode()}")
oddzielenie()

#-------------------------------------------------------
#Zmiana wartośći pola:
r.hincrby('pc_1', 'rok', 2)
dane = r.hgetall('pc_1')
print("Dane PC1 po aktualizacji: ")
for k, v in dane.items():
    print(f"{k.decode()} : {v.decode()}")
oddzielenie()

#-------------------------------------------------------
#Sprawdzanie, czy pole isntieje:
czyIstnieje1 = r.hexists('pc_1', 'model')
czyIstnieje2 = r.hexists('pc_1', 'RAM')

print("Pole 'model' istnieje" if czyIstnieje1 else "Brak pola 'model'!")
print("Pole 'RAM' istnieje" if czyIstnieje2 else "Brak pola 'RAM'!")
oddzielenie()

#-------------------------------------------------------
#Lista wszystkich kluczy i wartości:
print(f"Pola: {[k.decode() for k in r.hkeys('pc_1')]}")
print(f"Wartości: {[v.decode() for v in r.hvals('pc_1')]}")
oddzielenie()

#-------------------------------------------------------
#Usunięcie pola:
r.hdel('pc_1', 'rok')
print("Po usunięciu pola 'rok':")
for k, v in r.hgetall('pc_1').items():
    print(f"{k.decode()}: {v.decode()}")
oddzielenie()


Dane PC1: 
marka : Lenovo
model : TinkPad
rok : 2024
--------------------------------------------------
Marka PC1: Lenovo
--------------------------------------------------
Dane PC1 po aktualizacji: 
marka : Lenovo
model : TinkPad
rok : 2026
--------------------------------------------------
Pole 'model' istnieje
Brak pola 'RAM'!
--------------------------------------------------
Pola: ['marka', 'model', 'rok']
Wartości: ['Lenovo', 'TinkPad', '2026']
--------------------------------------------------
Po usunięciu pola 'rok':
marka: Lenovo
model: TinkPad
--------------------------------------------------


---
## Działania na zbiorach (Sets):

In [42]:
import redis as r
r = r.Redis('localhost', port=6379, db=0)

#-------------------------------------------------------
def oddzielenie():
    print("-" * 50)

#-------------------------------------------------------
# Czyścimy stare dane (tylko do testów):
r.delete('FC_Barcelona', 'Real_Madrid', 'reprezentacja_Brazylii')

#-------------------------------------------------------
# Dodajemy piłkarzy do zbioru reprezentującego klub FC Barcelona
r.sadd('FC_Barcelona', 'Lewandowski', 'Pedri', 'Fermin', 'Raphinha', 'Kounde')

# Dodajemy piłkarzy do zbioru reprezentującego klub Real Madryt
r.sadd('Real_Madrid', 'Vinicius', 'Modric', 'Kroos', 'Rudiger', 'Kounde')

# Dodajemy piłkarzy do zbioru reprezentacji Brazylii
r.sadd('reprezentacja_Brazylii', 'Vinicius', 'Raphinha', 'Alisson', 'Militao')

#-------------------------------------------------------
# Pobieramy i wypisujemy wszystkich piłkarzy z FC Barcelony
print("FC Barcelona:")
for zawodnik in r.smembers('FC_Barcelona'):
    print(f"- {zawodnik.decode()}")
oddzielenie()

#-------------------------------------------------------
# Piłkarze, którzy grają zarówno w Realu Madryt, jak i w reprezentacji Brazylii
# (część wspólna zbiorów: Real_Madrid ∩ reprezentacja_Brazylii)
wspolni = r.sinter('Real_Madrid', 'reprezentacja_Brazylii')
print("Piłkarze Realu grający też w reprezentacji Brazylii:")
for z in wspolni:
    print(f"- {z.decode()}")
oddzielenie()

#-------------------------------------------------------
# Piłkarze, którzy są w Barcelonie, ale nie w Realu Madryt
# (różnica zbiorów: FC_Barcelona - Real_Madrid)
roznica = r.sdiff('FC_Barcelona', 'Real_Madrid')
print("Piłkarze Barcelony, którzy nie grają w Realu:")
for z in roznica:
    print(f"- {z.decode()}")
oddzielenie()

#-------------------------------------------------------
# Wszyscy piłkarze grający w Barcelonie lub Realu (bez powtórzeń)
# (suma zbiorów: FC_Barcelona ∪ Real_Madrid)
wszyscy = r.sunion('FC_Barcelona', 'Real_Madrid')
print("Wszyscy piłkarze z obu klubów (bez powtórzeń):")
for z in wszyscy:
    print(f"- {z.decode()}")
oddzielenie()

#-------------------------------------------------------
# Sprawdzamy, czy konkretny zawodnik gra w reprezentacji Brazylii
# (czy element należy do zbioru: SISMEMBER)
zawodnik = 'Raphinha'
print(f"{zawodnik} gra w reprezentacji Brazylii?" +
      (" Tak!" if r.sismember('reprezentacja_Brazylii', zawodnik) else " Nie."))


FC Barcelona:
- Lewandowski
- Fermin
- Kounde
- Raphinha
- Pedri
--------------------------------------------------
Piłkarze Realu grający też w reprezentacji Brazylii:
- Vinicius
--------------------------------------------------
Piłkarze Barcelony, którzy nie grają w Realu:
- Lewandowski
- Fermin
- Raphinha
- Pedri
--------------------------------------------------
Wszyscy piłkarze z obu klubów (bez powtórzeń):
- Lewandowski
- Fermin
- Rudiger
- Kroos
- Vinicius
- Kounde
- Raphinha
- Pedri
- Modric
--------------------------------------------------
Raphinha gra w reprezentacji Brazylii? Tak!


---
## Działania na zbiorach uporządkowanych (Sorted Sets):

In [ ]:
import redis as r
r = r.Redis('localhost', port=6379, db=0)

def oddzielenie():
    print("-" * 50)

#-------------------------------------------------------
# Usuwanie wcześniej istniejących zbiorów (dla testu)
# Usuwa uporządkowane zbiory benchmarków NVIDIA i AMD (jeśli istnieją)
r.delete('benchmark_nvidia', 'benchmark_amd')

#-------------------------------------------------------
# Tworzenie uporządkowanego zbioru benchmarków kart NVIDIA
# ZADD – dodaje elementy do sorted set z przypisaną wydajnością
r.zadd('benchmark_nvidia', {
    'RTX 4090': 38500,
    'RTX 4080': 29500,
    'RTX 4070 Ti': 22500,
    'RTX 4060': 17500
})

# Tworzenie uporządkowanego zbioru benchmarków kart AMD
r.zadd('benchmark_amd', {
    'RX 7900 XTX': 35000,
    'RX 7900 XT': 30000,
    'RX 7800 XT': 23500,
    'RX 7600': 16000
})

#-------------------------------------------------------
# Odczyt kart NVIDIA posortowanych rosnąco wg benchmarku
# (Pobranie elementów od najniższego score do najwyższego):
print("Karty NVIDIA (benchmark rosnąco):")
for karta, wynik in r.zrange('benchmark_nvidia', 0, -1, withscores=True):
    print(f"{karta.decode()} - {int(wynik)} pkt")
oddzielenie()

#-------------------------------------------------------
# Odczyt kart AMD posortowanych malejąco wg benchmarku
# (Pobranie elementów od najwyższego score do najniższego):

print("Karty AMD (benchmark malejąco):")
for karta, wynik in r.zrevrange('benchmark_amd', 0, -1, withscores=True):
    print(f"{karta.decode()} - {int(wynik)} pkt")
oddzielenie()

#-------------------------------------------------------
# Sprawdzenie miejsca (rankingu) wybranej karty AMD (odczyt konkretnego pola):
ranking = r.zrevrank('benchmark_amd', 'RX 7800 XT')
print(f"RX 7800 XT jest na miejscu: {ranking + 1}")
oddzielenie()

#-------------------------------------------------------
# Zwiększanie score dla wybranej karty NVIDIA (zwiększanie wartości):
r.zincrby('benchmark_nvidia', 2000, 'RTX 4060')
print("Zwiększono wydajność RTX 4060 o 2000 punktów!")

# Wyświetlenie zaktualizowanego rankingu NVIDIA
print("Nowy ranking NVIDIA (po aktualizacji):")
for karta, wynik in r.zrevrange('benchmark_nvidia', 0, -1, withscores=True):
    print(f"{karta.decode()} - {int(wynik)} pkt")
oddzielenie()

#-------------------------------------------------------
# Usuwanie pola:
r.zrem('benchmark_amd', 'RX 7600')
print("RX 7600 została usunięta z rankingu AMD.")
oddzielenie()

#-------------------------------------------------------
# Tworzenie nowego uporządkowanego zbioru na podstawie połączenia dwóch innych (łączenie zbiorów):
r.zunionstore('benchmark_all', ('benchmark_nvidia', 'benchmark_amd'))

# Wyświetlenie połączonego rankingu wszystkich kart
print("Połączony ranking NVIDIA + AMD (benchmark):")
for karta, wynik in r.zrevrange('benchmark_all', 0, -1, withscores=True):
    print(f"{karta.decode()} - {int(wynik)} pkt")


---
## Działania na strumieniach (Stream's):

In [ ]:
import redis as r
r = r.Redis('localhost', port=6379, db=0)

def oddzielenie():
    print("-" * 50)

#-------------------------------------------------------
# Usuwanie wcześniej istniejącego strumienia (dla testu)
# Usuwamy strumień 'aktywnosc_uzytkownika', jeżeli istnieje, aby nie było konfliktów w trakcie testów.
r.delete('aktywnosc_uzytkownika')

#-------------------------------------------------------
# Tworzenie strumienia z aktywnościami użytkowników:
r.xadd('aktywnosc_uzytkownika', {
    'id': 'uzytkownik123',
    'akcja': 'logowanie',
    'czas': '2025-05-08T10:00:00'
})
r.xadd('aktywnosc_uzytkownika', {
    'id': 'uzytkownik124',
    'akcja': 'wylogowanie',
    'czas': '2025-05-08T10:05:00'
})
r.xadd('aktywnosc_uzytkownika', {
    'id': 'uzytkownik123',
    'akcja': 'wyslanie_wiadomosci',
    'czas': '2025-05-08T10:10:00'
})

#-------------------------------------------------------
# Odczyt wszystkich zdarzeń z strumienia(od najstarszego zdarzenia do najnowszego):
print("Aktywności użytkowników (wszystkie zdarzenia):")
for zdarzenie in r.xrange('aktywnosc_uzytkownika'):
    print(f"ID: {zdarzenie[0]} | Data: {zdarzenie[1]}")
oddzielenie()

#-------------------------------------------------------
# Odczyt zdarzeń od konkretnego momentu:
print("Aktywności użytkowników (od momentu):")
for zdarzenie in r.xrange('aktywnosc_uzytkownika', minid='1683540000000-0'):
    print(f"ID: {zdarzenie[0]} | Data: {zdarzenie[1]}")
oddzielenie()

#-------------------------------------------------------
# Czytanie nowych zdarzeń (strumień jako kanał):
# Używamy "0", aby zacząć odczyt od wszystkich zdarzeń.
print("Nowe aktywności użytkowników (odczyt na bieżąco):")
strumienie = r.xread({'aktywnosc_uzytkownika': '0'}, block=0)  # "0" oznacza wszystkie nowe zdarzenia
for strumien in strumienie:
    for zdarzenie in strumien[1]:
        print(f"Nowe zdarzenie: {zdarzenie[1]}")

#-------------------------------------------------------
# Usuwanie strumienia.
r.delete('aktywnosc_uzytkownika')
print("Strumień 'aktywnosc_uzytkownika' został usunięty.")

---
## Działania na bitmapach:

In [ ]:
import redis as r

r = r.Redis('localhost', port=6379, db=0)

def oddzielenie():
    print("-" * 50)

#-------------------------------------------------------
# Usuwanie istniejącej bitmapy obecności studenta (dla testu)
r.delete('obecnosc_studenta_789')

#-------------------------------------------------------
# Tworzenie bitmapy obecności dla studenta o ID 789
# 0 = poniedziałek, 1 = wtorek, ..., 4 = piątek

r.setbit('obecnosc_studenta_789', 0, 1)  # Poniedziałek - obecny
r.setbit('obecnosc_studenta_789', 1, 0)  # Wtorek - nieobecny
r.setbit('obecnosc_studenta_789', 2, 1)  # Środa - obecny
r.setbit('obecnosc_studenta_789', 3, 1)  # Czwartek - obecny
r.setbit('obecnosc_studenta_789', 4, 0)  # Piątek - nieobecny

#-------------------------------------------------------
# Sprawdzanie obecności studenta w poszczególne dni
dni = ['Poniedziałek', 'Wtorek', 'Środa', 'Czwartek', 'Piątek']
for i, dzien in enumerate(dni):
    status = r.getbit('obecnosc_studenta_789', i)
    print(f"Obecność studenta w {dzien}: {status}")

oddzielenie()

#-------------------------------------------------------
# Liczenie dni obecności studenta w tygodniu
obecnosc = r.bitcount('obecnosc_studenta_789')
print(f"Liczba dni obecności studenta na zajęciach w tygodniu: {obecnosc}")

oddzielenie()

#-------------------------------------------------------
# Dodanie obecności w dodatkowy dzień (np. sobota – zajęcia dodatkowe)
r.setbit('obecnosc_studenta_789', 5, 1)  # Sobota - obecny
print(f"Obecność studenta w Sobota: {r.getbit('obecnosc_studenta_789', 5)}")

oddzielenie()

#-------------------------------------------------------
# Usuwanie bitmapy po zakończeniu
r.delete('obecnosc_studenta_789')
oddzielenie()


---
## Działania na bitfield'ach:

In [ ]:
import redis

r = redis.Redis('localhost', port=6379, db=0)

def oddzielenie():
    print("-" * 50)

#-------------------------------------------------------
# Usuwamy istniejący bitfield (dla testu)
r.delete('statusy_zamowien')

#-------------------------------------------------------
# Ustawiamy statusy trzech zamówień (każdy status na 2 bitach)
# Zamówienie 1: status = 1 (w realizacji)
# Zamówienie 2: status = 2 (wysłane)
# Zamówienie 3: status = 0 (nowe)

r.bitfield('statusy_zamowien',
    'SET', 'u2', 0, 1,   # zamówienie 1 (offset 0)
    'SET', 'u2', 2, 2,   # zamówienie 2 (offset 2)
    'SET', 'u2', 4, 0    # zamówienie 3 (offset 4)
)

# Odczytujemy statusy zamówień
wyniki = r.bitfield('statusy_zamowien',
    'GET', 'u2', 0,
    'GET', 'u2', 2,
    'GET', 'u2', 4
)
print(f"Status zamówienia 1: {wyniki[0]}")
print(f"Status zamówienia 2: {wyniki[1]}")
print(f"Status zamówienia 3: {wyniki[2]}")
oddzielenie()

#-------------------------------------------------------
# Zmieniamy status zamówienia 1 na 'wysłane' (2)
r.bitfield('statusy_zamowien', 'SET', 'u2', 0, 2)
nowy_status = r.bitfield('statusy_zamowien', 'GET', 'u2', 0)[0]
print(f"Nowy status zamówienia 1: {nowy_status}")
oddzielenie()

#-------------------------------------------------------
# Usuwamy bitfield po zakończeniu
r.delete('statusy_zamowien')
oddzielenie()


---
## Działania na HyperLogLog:

In [ ]:
import redis

# Połączenie z Redis
r = redis.Redis(host='localhost', port=6379, db=0)

def oddzielenie():
    print("-" * 50)

#-------------------------------------------------------
# Usuwanie istniejącego HyperLogLog (dla testu)
r.delete('unikalne_pesel')

#-------------------------------------------------------
# Dodawanie numerów PESEL do HyperLogLog
r.pfadd('unikalne_pesel', '44051401359')  # Przykładowy PESEL
r.pfadd('unikalne_pesel', '79020813247')  # Przykładowy PESEL
r.pfadd('unikalne_pesel', '66031513789')  # Przykładowy PESEL
r.pfadd('unikalne_pesel', '44051401359')  # Duplikat (nie zostanie policzony)

#-------------------------------------------------------
# Sprawdzanie liczby unikalnych numerów PESEL
liczba_unikalnych_pesel = r.pfcount('unikalne_pesel')
print(f"Liczba unikalnych numerów PESEL: {liczba_unikalnych_pesel}")
oddzielenie()

#-------------------------------------------------------
# Dodawanie innych numerów PESEL do innego HyperLogLog
r.pfadd('unikalne_pesel_2', '85071574522')
r.pfadd('unikalne_pesel_2', '95051012345')
r.pfadd('unikalne_pesel_2', '79020813247')  # Duplikat

#-------------------------------------------------------
# Łączenie dwóch HyperLogLog (unifikacja wyników)
r.pfmerge('wszystkie_pesel', 'unikalne_pesel', 'unikalne_pesel_2')

#-------------------------------------------------------
# Sprawdzanie liczby unikalnych numerów PESEL po połączeniu
liczba_unikalnych_po_polaczeniu = r.pfcount('wszystkie_pesel')
print(f"Liczba unikalnych numerów PESEL po połączeniu: {liczba_unikalnych_po_polaczeniu}")
oddzielenie()


---
### Działania na danych geolokalizacyjnych (Geospatial):

In [45]:
import redis

# Połączenie z Redis
r = redis.Redis(host='localhost', port=6379, db=0)

#-------------------------------------------------------
def oddzielenie():
    print("-" * 50)

#-------------------------------------------------------
# Usuwamy istniejący zbiór geospatial (dla testu)
r.delete('miasta')

#-------------------------------------------------------
# Dodawanie lokalizacji miast (długość, szerokość geograficzna)
r.geoadd('miasta', (19.933, 53.917, 'Morąg'))
r.geoadd('miasta', (19.40884, 54.1522, 'Elbląg'))
r.geoadd('miasta', (19.667, 54.050, 'Pasłęk'))
r.geoadd('miasta', (18.61298, 54.37069, 'Gdańsk'))
r.geoadd('miasta', (20.46657, 53.77657, 'Olsztyn'))

#-------------------------------------------------------
# Sprawdzanie pozycji miast w zbiorze geospatial
wyniki = r.geopos('miasta', 'Morąg', 'Elbląg', 'Pasłęk', 'Gdańsk', 'Olsztyn')
for miasto, wspolrzedne in zip(['Morąg', 'Elbląg', 'Pasłęk', 'Gdańsk', 'Olsztyn'], wyniki):
    print(f"{miasto}: {wspolrzedne}")
oddzielenie()

#-------------------------------------------------------
# Obliczanie odległości między Morągiem a Gdańskiem
odleglosc = r.geodist('miasta', 'Morąg', 'Gdańsk', unit='km')
print(f"Odległość pomiędzy Morągiem a Gdańskiem: {odleglosc} km")
oddzielenie()

#-------------------------------------------------------
# Znajdowanie miast w promieniu 100 km od Morąga
miasta_w_promieniu = r.georadiusbymember('miasta', 'Morąg', 50, unit='km')
print("Miasta w promieniu 50 km od Morąga:")
for miasto in miasta_w_promieniu:
    print(miasto.decode())
oddzielenie()


Morąg: (19.932999908924103, 53.917000498651845)
Elbląg: (19.408842623233795, 54.152199809748055)
Pasłęk: (19.666999876499176, 54.04999985260352)
Gdańsk: (18.61297756433487, 54.37069023896191)
Olsztyn: (20.466571748256683, 53.776569342260984)
--------------------------------------------------
Odległość pomiędzy Morągiem a Gdańskiem: 99.7101 km
--------------------------------------------------
Miasta w promieniu 50 km od Morąga:
Morąg
Olsztyn
Elbląg
Pasłęk
--------------------------------------------------
